In [1]:
from __future__ import print_function, division
import pandas as pd, numpy as np
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook, tqdm
tqdm.pandas(tqdm_notebook(desc=''))

import time
from IPython.display import clear_output

/home/denis/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/denis/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
DATA_PATH = '' #Set your path here

In [3]:
#---------------------- Train --------------------------------
Car_train = pd.read_csv(DATA_PATH+'train/Car/Label/Car.csv')
Bus_train = pd.read_csv(DATA_PATH+'train/Bus/Label/Bus.csv')
Person_train = pd.read_csv(DATA_PATH+'train/Person/Label/Person.csv')
Truck_train = pd.read_csv(DATA_PATH+'train/Truck/Label/Truck.csv')
#----------------------- Test --------------------------------
Car_test = pd.read_csv(DATA_PATH+'test/Car/Label/Car.csv')
Bus_test = pd.read_csv(DATA_PATH+'test/Bus/Label/Bus.csv')
Person_test = pd.read_csv(DATA_PATH+'test/Person/Label/Person.csv')
Truck_test = pd.read_csv(DATA_PATH+'test/Truck/Label/Truck.csv')

In [4]:
#---------------------- TrainPath --------------------------------
Car_train['PATH'] = Car_train['ImageID'].apply(lambda x: 'train/Car/'+x+'.jpg')
Bus_train['PATH'] = Bus_train['ImageID'].apply(lambda x: 'train/Bus/'+x+'.jpg')
Person_train['PATH'] = Person_train['ImageID'].apply(lambda x: 'train/Person/'+x+'.jpg')
Truck_train['PATH'] = Truck_train['ImageID'].apply(lambda x: 'train/Truck/'+x+'.jpg')
#----------------------- TestPath --------------------------------
Car_test['PATH'] = Car_test['ImageID'].apply(lambda x: 'test/Car/'+x+'.jpg')
Bus_test['PATH'] = Bus_test['ImageID'].apply(lambda x: 'test/Bus/'+x+'.jpg')
Person_test['PATH'] = Person_test['ImageID'].apply(lambda x: 'test/Person/'+x+'.jpg')
Truck_test['PATH'] = Truck_test['ImageID'].apply(lambda x: 'test/Truck/'+x+'.jpg')
#----------------------- Labels ------------------------------
Car_train['CAT'], Car_test['CAT'] = 0, 0
Bus_train['CAT'], Bus_test['CAT'] = 1, 1
Person_train['CAT'], Person_test['CAT'] = 3, 3
Truck_train['CAT'], Truck_test['CAT'] = 2, 2
#---------------------- concatenate --------------------------
train = pd.concat((Car_train, Bus_train, Person_train, Truck_train))
test = pd.concat((Car_test, Bus_test, Person_test, Truck_test))

In [5]:
def convert_labels(xmin,xmax, ymin, ymax):
    """
    Definition: Parses label files to extract label and bounding box
        coordinates.  Converts (x1, y1, x1, y2) KITTI format to
        (x, y, width, height) normalized YOLO format.
    """
   
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    
    return x,y,w,h

def proceed(df):
    df['x'], df['y'], df['w'], df['h'] = zip(*df[['XMin', 'XMax', 'YMin', 'YMax']].progress_apply(lambda row: convert_labels(row['XMin'], row['XMax'],
                                                                                                                             row['YMin'], row['YMax']), axis = 1))
    return df

In [6]:
train = proceed(train)
test = proceed(test)

100%|██████████| 70920/70920 [00:01<00:00, 41598.30it/s]


In [12]:
def save_txt(name, text):
    with open(name, 'w') as f:
        f.write(text)

def make_img_txt(df, t = 'train'):
    path_change = []
    for category in df['CAT'].unique():
        df_cat = df[df['CAT']==category].copy()
        PATHES = df_cat['PATH'].unique()
        for path in tqdm_notebook((PATHES), total = len(PATHES), desc='Category: {}'.format(category)):
            # Make sure that category doesn't cross
            path_change.append(path)
            text = ''
            for label in df_cat[df_cat['PATH'] == path][['CAT', 'x', 'y', 'w', 'h']].astype(str).values:
                text += ' '.join(label) + '\n'
            save_txt(DATA_PATH+path.split('.')[0] + '.txt', text)
    np.savetxt('data/'+t+ '.txt', path_change, fmt='%s')

#For pool/threads testing
def make_img_txt_solo(df, category):
    df_cat = df[df['CAT']==category].copy()
    PATHES = df_cat['PATH'].unique()
    for path in tqdm((PATHES), total = len(PATHES), desc='Category: {}'.format(category)):
        # Make sure that category doesn't cross
        text = ''
        for label in df_cat[df_cat['PATH'] == path][['CAT', 'x', 'y', 'w', 'h']].astype(str).values:
            text += ' '.join(label) + '\n'
        save_txt(DATA_PATH+path.split('.')[0] + '.txt', text)
    return None

In [ ]:
from multiprocessing.pool import Pool
from functools import partial

pool = Pool(4)
func = partial(make_img_txt_solo, train)
list(tqdm(pool.imap(func, train['CAT'].unique()), total = len(train['CAT'].unique())))
pool.close()
pool.join()

Category: 3:   9%|▊         | 21176/248384 [15:18<2:44:13, 23.06it/s]

In [15]:
np.savetxt('data/'+'train'+ '.txt', train['PATH'].unique(), fmt='%s')

In [13]:
make_img_txt(test, 'test')